In [1]:
from toxicDetector import *
from sentiment_new import *
import time

In [2]:
def get_useful_words(n,fea_names,coeffs):
    '''
    get importtant words from feature names given cls.coefficients

    Args:
        fea_names = sentiment.count_vect.get_feature_names()
        coeffs = cls.coef_[0]
    Return:
        List of important words

    '''
    assert (n%2==0) and isinstance(n,int)
    k=int(n/2)
    sortword =np.argsort(coeffs)
    top_k= sortword[-k:]
    bottom_k=sortword[:k]
    toxic_words=[]
    nontoxic_words=[]
    for i in top_k:
        toxic_words.append(fea_names[i])

    for i in bottom_k:
        nontoxic_words.append(fea_names[i])

    return toxic_words , nontoxic_words

In [3]:
time_start = time.time()
print("Reading data")
maxdf = 0.32
mindf = 1
sentiment = sentimentinterface.read_files( min_df = mindf, max_df = maxdf)
solve_name = 'sag'
penalty = 'l2'
print("finished reading data")
print("start training data")
cls = train_classifier(sentiment.trainX, sentiment.trainy, penalty = penalty, solver = solve_name)
print("Classifier trained")

coef_now = cls.coef_[0]
fea_names = sentiment.count_vect.get_feature_names()
coeffs = cls.coef_[0]
print('shape of coef_pre:' ,coef_now.shape)

toxic_words, nontoxic_words = get_useful_words(200,fea_names,coef_now)
print("________USEFUL WORDS_________")
print("toxic word list: ", toxic_words)
print("nontoxic word list: ", nontoxic_words)


print("start evaluating")
acc = evaluate(sentiment.devX, sentiment.devy, cls, 'dev data')
print("finished evaluation")
print('Accuracy before using stopwords: {}'.format(acc))

time_elapsed = time.time() - time_start
print("time used: {}".format(time_elapsed))

Reading data
-- train data
training dataset amount: 165496
toxic in train set: 16098  (9.73%)
non-toxic in train set : 149398
-- dev data
dev dataset amount: 10560 
toxic in dev set : 1065  (10.09%)
non-toxic in dev set : 9495  
finished reading data
start training data
Classifier trained
shape of coef_pre: (1550282,)
________USEFUL WORDS_________
toxic word list:  ['you you', 'grow', 'yourself', 'hypocrite', 'balls', 'jews', 'an asshole', 'piece', 'are an', 'blah', 'stop being', 'garbage', 'assholes', 'troll', 'spam', 'your', 'don be', 'mother', 'lies', 'stupidity', 'bunch of', 'the hell', 'piece of', 'morons', 'face', 'dumbass', 'and you', 'mouth', 'fuckin', 'scum', 'bunch', 'you suck', 'retard', 'fucker', 'bloody', 'little', 'piss', 'of shit', 'you are', 'dirty', 'disgusting', 'pussy', 'me you', 'homosexual', 'shut up', 'nigger', 'ridiculous', 'fucked', 'like you', 'wtf', 'what the', 'fuck off', 'sick', 'idiotic', 'kill', 'fag', 'an idiot', 'jerk', 'fat', 'off', 'retarded', 'racist'

In [5]:
stopwords = get_stopwords(1000000,fea_names,coef_now)

## Testing by random input

In [26]:
input_string = ["go to the hell you son of bitch"," i love u",'you are so annoying','I don\'t really like you','how do you think about Eric','I want to kick him',""]
input_vect = sentiment.count_vect.transform(input_string)
probs = cls.predict_proba(input_vect)
label = cls.predict(input_vect)
label_sentiment = sentiment.le.inverse_transform(label)

label_list = [x[1] for x in probs]

for i in range(len(input_string)):
    print('{} : {}   ({}) '.format(input_string[i], label_list[i], label_sentiment[i]))

go to the hell you son of bitch : 0.9389627121725953   (toxic) 
 i love u : 0.27079326303185913   (peace) 
you are so annoying : 0.4810936704028729   (peace) 
I don't really like you : 0.43936569993927754   (peace) 
how do you think about Eric : 0.2448487309278115   (peace) 
I want to kick him : 0.3268353833919887   (peace) 
 : 0.3121504881712093   (peace) 


In [9]:
print(len(fea_names))
coef_now[0]


1550282


-0.06276225559135505

In [ ]:
import csv
'''
f = open('train.tsv', 'w')
for i in range(len(data_train)):
    f.write('peace' if labels_train[i] is False else 'toxic')
    f.write("\t")
    f.write(data_train[i])
    f.write("\n")
f.close()

f = open('dev.tsv', 'w')
for i in range(len(data_dev)):
    f.write('peace' if labels_dev[i] is False else 'toxic')
    f.write("\t")
    f.write(data_dev[i])
    f.write("\n")
f.close()
'''

f = open('train.csv', 'a')
writer = csv.writer(f)
for i in range(len(data_train)):
    l = 'peace' if labels_train[i] is False else 'toxic'
    text = data_train[i]
    row = [l,text]
    writer.writerow(row)
f.close()

f = open('dev.csv', 'a')
writer = csv.writer(f)
for i in range(len(data_dev)):
    l = 'peace' if labels_dev[i] is False else 'toxic'
    text = data_dev[i]
    row = [l,text]
    writer.writerow(row)
f.close()

In [ ]:
#### Save the cls object into pickle

In [27]:
import pickle
output = open("basic_toxicDetector.pkl",'wb')
pickle.dump(cls, output)
output.close()

In [12]:
pkl_input = open("basic_toxicDetector.pkl", 'rb')
cls1 = pickle.load(pkl_input)
pkl_input.close()

In [ ]:
def read_csv(fname):
    data = []
    labels = []
    f = open(fname, 'r')
    #readCSV = csv.reader(f, delimiter = ',')
    
    reader = csv.reader(f)
    for row in reader:
        labels.append(row[0])
        data.append(row[1])
    return data, labels

d, l = read_csv('train.csv')